In [1]:
N = ord("D") % 5 + 1
N

4

In [9]:
import cv2 as cv
from PIL import Image, ImageDraw
import numpy as np

face_cascade = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_eye.xml')

def process_image(input_image_path, output_image_path, glasses_color='magenta', line_width=5):
    image = cv.imread(input_image_path)
    gray_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    for (x, y, w, h) in faces:
        roi_gray = gray_image[y:y+h, x:x+w]
        roi_color = image[y:y+h, x:x+w]

        eyes = eye_cascade.detectMultiScale(roi_gray, scaleFactor=1.1, minNeighbors=10, minSize=(20, 20))

        if len(eyes) >= 2:
            pil_image = Image.fromarray(cv.cvtColor(image, cv.COLOR_BGR2RGB))
            draw = ImageDraw.Draw(pil_image)

            eyes = sorted(eyes, key=lambda e: e[0])
            eye_centers = [
                (x + ex + ew // 2, y + ey + eh // 2)
                for (ex, ey, ew, eh) in eyes[:2]
            ]

            eye_size = max(eyes[0][2], eyes[0][3], eyes[1][2], eyes[1][3])
            distance = eye_centers[1][0] - eye_centers[0][0]
            glasses_width, glasses_height = distance + eye_size, eye_size

            draw.ellipse(
                [eye_centers[0][0] - glasses_width // 4, eye_centers[0][1] - glasses_height // 2,
                 eye_centers[0][0] + glasses_width // 4, eye_centers[0][1] + glasses_height // 2],
                outline=glasses_color, width=line_width
            )

            draw.ellipse(
                [eye_centers[1][0] - glasses_width // 4, eye_centers[1][1] - glasses_height // 2,
                 eye_centers[1][0] + glasses_width // 4, eye_centers[1][1] + glasses_height // 2],
                outline=glasses_color, width=line_width
            )

            draw.line(
                [(
                    eye_centers[0][0] + glasses_width // 4, eye_centers[0][1]),
                 (eye_centers[1][0] - glasses_width // 4, eye_centers[1][1])],
                fill=glasses_color, width=line_width
            )

            cv_image = cv.cvtColor(np.array(pil_image), cv.COLOR_RGB2BGR)

            face_center = (x + w // 2, y + h // 2)
            crop_x1, crop_y1 = max(face_center[0] - 150, 0), max(face_center[1] - 150, 0)
            crop_x2, crop_y2 = min(face_center[0] + 150, cv_image.shape[1]), min(face_center[1] + 150, cv_image.shape[0])
            cropped = cv_image[crop_y1:crop_y2, crop_x1:crop_x2]
            resized = cv.resize(cropped, (600, 600), interpolation=cv.INTER_AREA)

            cv.imwrite(output_image_path, resized)

input_image_path = "images/longbottom.jpg"
output_image_path = "images/longbottom_final.jpg"
process_image(input_image_path, output_image_path)